In [1]:
import pandas as pd

In [2]:
otomoto_data = pd.read_pickle('data\otomoto_data.pickle')

In [3]:
otomoto_data

,price,Rok produkcji,Przebieg,Pojemność skokowa,Moc,Liczba drzwi,Liczba miejsc,ABS,Aktywne rozpoznawanie znaków ograniczenia prędkości,Aktywny asystent hamowania awaryjnego,...,Kolor_Szary,Kolor_Zielony,Kolor_Złoty,Kolor_Żółty,Oferta od_Firmy,Oferta od_Osoby prywatnej,Skrzynia biegów_Automatyczna,Skrzynia biegów_Manualna,Stan_Nowe,Stan_Używane
0,39900.0,2009,158000.0,1368.0,215.0,3.0,5.0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
1,72900.0,2019,66500.0,1368.0,145.0,3.0,4.0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
2,39900.0,2012,193631.0,1400.0,135.0,3.0,4.0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
3,43900.0,2015,139000.0,1368.0,160.0,3.0,4.0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
4,36500.0,2013,103000.0,1368.0,160.0,3.0,4.0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130794,55800.0,2016,93000.0,1199.0,130.0,5.0,5.0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
130795,85000.0,2018,44000.0,1199.0,130.0,5.0,5.0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
130796,39999.0,2017,53000.0,1199.0,130.0,5.0,5.0,1,1,0,...,0,1,0,0,0,1,0,1,0,1
130797,32800.0,2013,133800.0,1560.0,112.0,5.0,5.0,1,0,0,...,0,0,0,0,1,0,0,1,0,1


In [4]:
X = otomoto_data.loc[:, otomoto_data.columns != 'price']
Y = otomoto_data['price']

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

SKLearn model

In [7]:
lr = LinearRegression()
lr.fit(X_train, Y_train)

c:\Users\micha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(


LinearRegression()

In [8]:
preds = lr.predict(X_test)

c:\Users\micha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(


In [9]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


print('MAE SKLearn:' + str(mean_absolute_error(Y_test, preds)))
print('RMSE SKLearn: ' + str(mean_squared_error(Y_test, preds, squared=False)))

MAE SKLearn:15993.187607259208
RMSE SKLearn: 26919.617440317255


In [10]:
import pickle
pickle.dump(lr, open('models/model_skl.pickle', 'wb'))

XGBoost model

In [11]:
import xgboost as xgb
import numpy as np

In [12]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.1, max_depth = 10)

In [13]:
xg_reg.fit(X_train,Y_train)

preds_xgb = xg_reg.predict(X_test)

In [14]:
rmse_xgb = np.sqrt(mean_squared_error(Y_test, preds_xgb))
mae_xgb = mean_absolute_error(Y_test, preds_xgb)
print(f"RMSE XGBoost: {rmse_xgb}")
print(f"MAE XGBoost: {mae_xgb}")

RMSE XGBoost: 18783.500319720755
MAE XGBoost: 9217.415294373553


In [15]:
xg_reg.save_model("models/model_xgb.txt")

In [30]:
train = xgb.DMatrix(X_train, Y_train)
test = xgb.DMatrix(X_test, Y_test)

# We need to define parameters as dict
params = {
    "learning_rate": 0.1,
    "max_depth": 10
}
# training, we set the early stopping rounds parameter
model_xgb = xgb.train(params, 
          train, evals=[(train, "train"), (test, "validation")], 
          num_boost_round=500, early_stopping_rounds=20)

[0]	train-rmse:75846.85235	validation-rmse:76423.79666
[1]	train-rmse:69008.62637	validation-rmse:69629.58089
[2]	train-rmse:62874.26502	validation-rmse:63600.53148
[3]	train-rmse:57381.58776	validation-rmse:58200.69627
[4]	train-rmse:52470.59372	validation-rmse:53390.86943
[5]	train-rmse:48073.62452	validation-rmse:49149.79954
[6]	train-rmse:44159.74376	validation-rmse:45347.79333
[7]	train-rmse:40626.14928	validation-rmse:41952.85971
[8]	train-rmse:37510.42721	validation-rmse:39021.06563
[9]	train-rmse:34721.22880	validation-rmse:36428.01778
[10]	train-rmse:32226.85757	validation-rmse:34172.84636
[11]	train-rmse:30036.11871	validation-rmse:32190.11707
[12]	train-rmse:28080.29630	validation-rmse:30458.08527
[13]	train-rmse:26370.05490	validation-rmse:28945.56359
[14]	train-rmse:24820.02536	validation-rmse:27588.15845
[15]	train-rmse:23502.88836	validation-rmse:26457.06564
[16]	train-rmse:22297.14521	validation-rmse:25495.02442
[17]	train-rmse:21281.77970	validation-rmse:24666.94081
[1

In [29]:
model_xgb.best_ntree_limit


100

In [ ]:
preds_xgboost = model_xgb.predict(test, ntree_limit=model_xgb.best_ntree_limit)


In [34]:
rmse_xgb = np.sqrt(mean_squared_error(Y_test, preds_xgboost))
mae_xgb = mean_absolute_error(Y_test, preds_xgboost)
print(f"RMSE XGBoost: {rmse_xgb}")
print(f"MAE XGBoost: {mae_xgb}")

RMSE XGBoost: 18302.630966275294
MAE XGBoost: 8771.714235467713


In [36]:
model_xgb.save_model("models/model_xgboost_ntree.json")
